In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten
from keras.preprocessing.image import ImageDataGenerator
import keras.backend.tensorflow_backend as K
from tensorflow.python.client import device_lib

Using TensorFlow backend.


In [2]:
print(device_lib.list_local_devices())

np.random.seed(3)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13646094010321245865
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4854028697
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12362784569915296249
physical_device_desc: "device: 0, name: GeForce GTX 1660 Ti, pci bus id: 0000:1f:00.0, compute capability: 7.5"
]


## 7번 문제

Q. 은하를 분류함에 있어서 RGB 조건이 반드시 반영되어야 할까?
A. No. Edge/Smooth/Spiral 은하가 분류되는 기준은 색상이 아닌 형태이기 때문에 grayscale로도 충분히 이 형태를 구분할 수 있다.

Q. 더 높은 정확도를 갖는 모델 설계하시오.
A . 좀 더 Convolution layer를 거치면 더 중요한 feature들을 추출해 낼 수 있기 때문에 

```python
model.add(Conv2D(128,(3,3),strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
```
layer를 넣어줬다. 그랬더니 Eval 결과가 70%대에서 81%로 올랐다.


In [7]:
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './train_galaxy', #파일 경로
        target_size=(50,50),
        color_mode="grayscale",
        batch_size=32,
        class_mode='categorical')

Found 10233 images belonging to 3 classes.


In [8]:
val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
        './val_galaxy', #파일 경로
        target_size=(50,50),
        color_mode="grayscale",
        batch_size=32,
        class_mode='categorical')

Found 600 images belonging to 3 classes.


In [9]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        './test_galaxy', #파일 경로
        target_size=(50,50),
        color_mode="grayscale",
        batch_size=32,
        class_mode='categorical')

Found 600 images belonging to 3 classes.


In [12]:
with K.tf.device('/gpu:0'):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(50,50,3)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(128,(3,3),strides=1, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(64,(3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    hist=model.fit_generator(
        train_generator,
        steps_per_epoch=319, 
        epochs=50,
        validation_data=val_generator,
        validation_steps=200)  

Epoch 1/50
319/319 [==============================] - 8s 26ms/step - loss: 0.6978 - accuracy: 0.7003 - val_loss: 0.6363 - val_accuracy: 0.8203
Epoch 2/50
319/319 [==============================] - 8s 26ms/step - loss: 0.5350 - accuracy: 0.7839 - val_loss: 0.4845 - val_accuracy: 0.8124
Epoch 3/50
319/319 [==============================] - 8s 25ms/step - loss: 0.4895 - accuracy: 0.8074 - val_loss: 0.5068 - val_accuracy: 0.8225
Epoch 4/50
319/319 [==============================] - 8s 26ms/step - loss: 0.4531 - accuracy: 0.8204 - val_loss: 0.5902 - val_accuracy: 0.8519
Epoch 5/50
319/319 [==============================] - 8s 26ms/step - loss: 0.4101 - accuracy: 0.8346 - val_loss: 0.4234 - val_accuracy: 0.8388
Epoch 6/50
319/319 [==============================] - 9s 27ms/step - loss: 0.3770 - accuracy: 0.8516 - val_loss: 0.7007 - val_accuracy: 0.8332
Epoch 7/50
319/319 [==============================] - 8s 26ms/step - loss: 0.3405 - accuracy: 0.8639 - val_loss: 0.3332 - val_accuracy: 0.8259

In [13]:
print("--Evaluate--")
scores = model.evaluate_generator(test_generator, steps=200)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

--Evaluate--
accuracy: 81.19%
